In [20]:
# source: https://www.youtube.com/watch?v=6nQlxJvcTr0
# hymenoptera dataset from: https://www.kaggle.com/ajayrana/hymenoptera-data

In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets

In [19]:
def set_parameter_requires_grad(model, extracting):
    if extracting:
        for param in model.parameters():
            param.requires_grad = False
            
def train_model(model, data_loaders, criterion, optimizer, epochs = 25):
    for epoch in range(epochs):
        print("Epoch %d / %d" %(epoch, epochs -1))
        print("-"*15)
        
        for phase in ["train","val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            correct = 0
            
            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / len(data_loaders[phase].dataset)
            epoch_acc = correct.double() / len(data_loaders[phase].dataset)
            
            print('{} Loss {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
if __name__ == "__main__":
    root_dir = "hymenoptera_data/"
    
    image_transforms = {
        "train":    transforms.Compose([transforms.RandomRotation((-270,270)),
                    transforms.Resize((224,224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])]),
        "val":      transforms.Compose([transforms.RandomRotation((-270,270)),
                    transforms.Resize((224,224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])])
    }
    
    data_generator = {k: datasets.ImageFolder(os.path.join(root_dir, k),
                                             image_transforms[k]) for k in ["train", "val"]}
    
    data_loader = {k: torch.utils.data.DataLoader(data_generator[k], batch_size = 2,
                                                shuffle = True, num_workers = 4) for k in ["train", "val"]}

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = models.resnet18(pretrained = True)
    
    set_parameter_requires_grad(model, True)
    
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad is True:
            params_to_update.append(param)
            print("\t", name)
            
    train_model(model, data_loader, criterion, optimizer)
    
            

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\lutha/.cache\torch\hub\checkpoints\resnet18-5c106cde.pth



	 fc.weight
	 fc.bias
Epoch 0 / 24
---------------
train Loss 0.7819 Acc: 0.5369
val Loss 0.7407 Acc: 0.5490
Epoch 1 / 24
---------------
train Loss 0.6915 Acc: 0.6352
val Loss 0.3737 Acc: 0.8431
Epoch 2 / 24
---------------
train Loss 0.6167 Acc: 0.6311
val Loss 0.4611 Acc: 0.7908
Epoch 3 / 24
---------------
train Loss 0.5775 Acc: 0.6762
val Loss 0.4384 Acc: 0.7908
Epoch 4 / 24
---------------
train Loss 0.5333 Acc: 0.7008
val Loss 0.4234 Acc: 0.8105
Epoch 5 / 24
---------------
train Loss 0.5498 Acc: 0.6967
val Loss 0.3207 Acc: 0.8758
Epoch 6 / 24
---------------
train Loss 0.5836 Acc: 0.7049
val Loss 0.3369 Acc: 0.8889
Epoch 7 / 24
---------------
train Loss 0.6682 Acc: 0.6598
val Loss 0.3001 Acc: 0.8758
Epoch 8 / 24
---------------
train Loss 0.5910 Acc: 0.6885
val Loss 0.3816 Acc: 0.8301
Epoch 9 / 24
---------------
train Loss 0.5934 Acc: 0.7418
val Loss 0.2944 Acc: 0.8954
Epoch 10 / 24
---------------
train Loss 0.5851 Acc: 0.6885
val Loss 0.3210 Acc: 0.8562
Epoch 11 / 24
-----